# Bases de connaissance : interroger Wikidata à l'aide de requêtes SPARQL

## Imports

In [1]:
from datetime import datetime as dt
from SPARQLWrapper import SPARQLWrapper, JSON

## Obtenir la liste des politiciens belges

In [3]:
# Retrieve results from SPARQL
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql = SPARQLWrapper(endpoint)

# P27 = country of citizenship 
# Q31 = Belgium
# P106 = Opccupation
# Q82955 = Politician
# P569 = date of birth
# P570 = date of death
statement = """
SELECT ?theme ?auteur ?titre ?date  WHERE {
    ?article wdt:P31 wd:Q191067 .
    ?article wdt:P1476 ?titre .
    ?article wdt:P577 ?date .
    ?article wdt:P921 ?theme .
    ?article wdt:P50 ?auteur.
   FILTER (?date >= "2022-04-23T00:00:00Z"^^xsd:dateTime)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY DESC(?date)
LIMIT 10
"""

sparql.setQuery(statement)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

rows = results['results']['bindings']
print(f"\n{len(rows)} article found\n")
print(rows[:10])


10 article found

[{'theme': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q167555'}, 'auteur': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q28530149'}, 'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '2023-07-12T00:00:00Z'}, 'titre': {'xml:lang': 'en', 'type': 'literal', 'value': 'BioHackJP 2023 Report R3: Expand the pathway analysis environment to non-model organisms'}}, {'theme': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q7999828'}, 'auteur': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q28530149'}, 'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '2023-07-12T00:00:00Z'}, 'titre': {'xml:lang': 'en', 'type': 'literal', 'value': 'BioHackJP 2023 Report R3: Expand the pathway analysis environment to non-model organisms'}}, {'theme': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q167555'}, 'auteur': {'type': 'uri', 'value': 'http://www.

## Filtrer pour n'afficher que les noms contenant "Alphonse"

In [ ]:
name_filter = 'Alphonse'
max_results = 10

date_format = "%Y-%m-%dT%H:%M:%SZ"
filtered_rows = [row for row in rows if name_filter in row['personLabel']['value']]
print(f"Displaying the first {max_results}:\n")
for row in filtered_rows[:max_results]:
    try:
        birth_date = dt.strptime(row['dateBirth']['value'], date_format)
        birth_year = birth_date.year
    except ValueError:
        birth_year = "????"
    try:
        death_date = dt.strptime(row['dateDeath']['value'], date_format)
        death_year = death_date.year
    except ValueError: # unknown death date
        death_year = "????"
    except KeyError: # still alive
        death_year = ""
    print(f"{row['personLabel']['value']} ({birth_year}-{death_year})")

## Pour en savoir plus

- [Le projet Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page)
- [Aide à la construction de requêtes](https://query.wikidata.org/)